In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv("train_v3.csv", index_col="Datetime")

In [5]:
df_test = pd.read_csv("test_v3.csv", index_col="Datetime")

In [6]:
scaler = StandardScaler()
scaler.fit(df["Category_1"])
df_test["Category_1"] = scaler.transform(df_test["Category_1"])

/home/iamnewneo/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/iamnewneo/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [7]:
df_test = df_test.drop(["Unnamed: 0"], axis=1)

df_test = df_test.drop(["Item_ID"], axis=1)


for i in range(1,7):
    df_test["month_"+str(i)] = np.zeros((df_test.shape[0]),int)

columns = df_test.columns.tolist()

In [8]:
for i in range(1,3):
    df_test["quarter_"+str(i)] = np.zeros((df_test.shape[0]),int)

In [9]:
columns = df_test.columns.tolist()

In [10]:
columns = [columns[1]] + [columns[0]] + columns[2:]

In [11]:
columns = columns[:18] + columns[-8:-2] + columns[18:25] + columns[-2:] + columns[-10:-8]

In [12]:
new_cols = columns

df_test = df_test[new_cols]

In [13]:
new_cols

['Category_3',
 'Category_1',
 'ID',
 'DayOfMonth',
 'DayCount',
 'cat_2_1.0',
 'cat_2_2.0',
 'cat_2_3.0',
 'cat_2_4.0',
 'cat_2_5.0',
 'cat_2_other',
 'week_0',
 'week_1',
 'week_2',
 'week_3',
 'week_4',
 'week_5',
 'week_6',
 'month_1',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_10',
 'month_11',
 'month_12',
 'is_weekend',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4']

In [14]:
import xgboost as xgb

/home/iamnewneo/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
sales_model = xgb.Booster()
sales_model.load_model("./models/sales_model_train_v4.model")

price_model = xgb.Booster()
price_model.load_model("./models/price_model_train_v3.model")

In [16]:
sales_pred = sales_model.predict(xgb.DMatrix(df_test.drop(["ID"],axis=1)))

In [17]:
df_test["Number_Of_Sales"] = sales_pred

In [18]:
# cols = df_test.columns.tolist()

In [19]:
# cols = cols[:2] + [cols[-1]] + cols[2:-1]

In [20]:
# df_test = df_test[cols]

In [21]:
price_pred = price_model.predict(xgb.DMatrix(df_test.drop(["ID"],axis=1)))

In [22]:
df_test["Price"] = price_pred

In [23]:
submission = df_test[["ID", "Number_Of_Sales", "Price"]]

In [24]:
submission.to_csv("iamnewneo_submission_v8.csv", index=False)